In [2]:
import sys
sys.dont_write_bytecode = True

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas

import matplotlib.pyplot as plt


# Local imports.
from resale_flat_prices.resale_flat_data.resale_flat_data import ResaleFlatData
from resale_flat_prices.resale_flat_data.rent_prices_data import RentPricesData
from resale_flat_prices.h3_utils.h3_statistics import monthly_median_price
from resale_flat_prices.vis_utils.vis_utils import plot_df


# Data directories.
csv_data_dir = Path("../data/ResaleFlatPrices/")
processed_data_dir = Path("../data/processed_data/")

In [41]:
# Load processed and geocoded resale flat data.
resale_flat_data_csv_file = "resale-flat-prices.csv.zip"

# H3 cell creation using a specified resolution.
RESOLUTION = 9

# Calculate the median prices.
datetime_column = "year_quarter"
wanted_datetimes = [
    "2024-Q4", "2024-Q3", 
    # "2024-Q2", "2024-Q1"
]
price_column = "price_per_sqft"

# Display columns.
disp_columns = [
    "year_month", "town", "block", "street_name", "flat_type", "storey_range", "floor_area_sqm", "age",
    "resale_price", "price_per_sqft", "h3"
]

resale_flat_data = ResaleFlatData(processed_data_dir / resale_flat_data_csv_file)
resale_flat_data.read_csv()
resale_flat_data.df = resale_flat_data.df.sort_values(["year_month", "town"])

# Create additional datetime columns indicating the quarter the resale occured in.
resale_flat_data.df["quarter"] = resale_flat_data.df["month"].apply(lambda x: int(np.ceil(x / 3)))
resale_flat_data.df["year_quarter"] = resale_flat_data.df.apply(lambda DF: str(DF["year"]) + "-Q" + str(DF["quarter"]), axis = 1)

# Restrict the resale dateetimes to prevent processing of unneeded data.
#resale_flat_data.df = resale_flat_data.df[resale_flat_data.df[datetime_column].isin(wanted_datetimes)]

print("Resale flat data DataFrame: {}.".format(resale_flat_data.df.shape))

Resale flat data DataFrame: (941344, 25).


In [45]:
df = resale_flat_data.df.copy()

#df = df[df["street_name_cleaned"] == "ANG MO KIO AVENUE 1"]
df = df[df['address'] == '310A ANG MO KIO AVENUE 1']

In [47]:
disp_cols = ['year', 'month', 'address', 'storey_range', 'flat_type', 'price_per_sqft']

df[disp_cols].sort_values(["year", "month", "price_per_sqft"], ascending = False)

,year,month,address,storey_range,flat_type,price_per_sqft
190886,2024,10,310A ANG MO KIO AVENUE 1,19 TO 21,5 ROOM,998
188676,2024,9,310A ANG MO KIO AVENUE 1,10 TO 12,5 ROOM,959
186079,2024,8,310A ANG MO KIO AVENUE 1,04 TO 06,5 ROOM,831
175985,2024,4,310A ANG MO KIO AVENUE 1,01 TO 03,4 ROOM,731
171836,2024,2,310A ANG MO KIO AVENUE 1,10 TO 12,5 ROOM,844
165077,2023,11,310A ANG MO KIO AVENUE 1,04 TO 06,4 ROOM,761
141219,2022,12,310A ANG MO KIO AVENUE 1,01 TO 03,4 ROOM,729
139095,2022,11,310A ANG MO KIO AVENUE 1,25 TO 27,4 ROOM,793
137139,2022,10,310A ANG MO KIO AVENUE 1,13 TO 15,5 ROOM,765
132288,2022,8,310A ANG MO KIO AVENUE 1,07 TO 09,5 ROOM,718
